In [3]:
import math
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [4]:
data = pd.read_parquet("data/raw/france.parquet")
data.dropna(axis=0, how='any', inplace=True)

position = pd.read_csv("data/raw/postesSynop.csv", sep=";")

Id = position["ID"].astype(str)
for i in range(len(Id)):
    if len(Id[i]) < 5:
        Id[i] = '0' + Id[i]

production = pd.read_parquet("data/raw/franceagrimer-rdts-surfs-multicrops.parquet")
production = production.drop(production[production["n_dep"] == "2A"].index)
production = production.drop(production[production["n_dep"] == "2B"].index)
production = production.drop(production[production["n_dep"].astype(int) > 95].index)

provinces = {7005: 80, 7015: 59, 7020: 50, 7027: 14, 7037: 76, 
             7072: 51, 7110: 29, 7117: 22, 7130: 35, 7139: 61, 
             7149: 91, 7168: 10, 7181: 54, 7190: 67, 7207: 56, 
             7222: 44, 7240: 37, 7255: 18, 7280: 21, 7299: 68, 
             7314: 17, 7335: 86, 7434: 87, 7460: 63, 7471: 43, 
             7481: 69, 7510: 33, 7535: 46, 7558: 12, 7577: 26, 
             7591: 5,  7607: 40, 7621: 65, 7627: 9,  7630: 31, 
             7643: 34, 7650: 13, 7661: 83, 7690: 6,  7747: 66, 67005: 10}

crops = production["crop"].unique()
stations = data["id_sta"].unique()

In [5]:
lr = LinearRegression()

# Single Element Regression by Month

In [6]:
def init_x(data_list, consider_name):
    for i in stations:
        if i in provinces:
            data_station = data[data["id_sta"] == i]
            day_position = 0

            for j in range(2017, 2023):
                for m in range(1, 13):
                    one_data = 0
                    if data_station.index[day_position].year == j:
                        if data_station.index[day_position].month == m:
                            for d in range(day_position, len(data_station)):
                                if data_station.index[d].month == m:
                                    one_data += data_station[consider_name][d]
                                else:
                                    day_position = d
                                    break
                            data_list[str(provinces[i]) + "_" + str(j) + "_" + str(m)] = one_data


def init_y(data_list):
    for i in data_list:
        n_dep0, year, month = i.split('_')
        if n_dep0 + "_" + str(year) not in total_rdt:
            r_year = production['n_dep'].map(lambda x: x == n_dep0)
            r_crop = production['crop'].map(lambda x: x == crop)
            rdt = production[r_year & r_crop]["rdt_" + str(year)].values
            if rdt.size > 0:
                if rdt[0]:
                    total_rdt[n_dep0 + "_" + str(year)] = rdt[0]


def init_list(data_list, month):
    temp_data_list = []
    temp_rdt_list = []

    for i in total_rdt:
        if i + "_" + str(month) in data_list:
            temp_data_list.append(np.array([data_list[i + "_" + str(month)]]))
            temp_rdt_list.append(total_rdt[i])

    temp_rdt_list = np.array(temp_rdt_list)

    return temp_data_list, temp_rdt_list


def add_degreed_data(data_list, degree):
    temp_data_list = data_list.copy()
    
    for i in range(len(data_list)):
        temp_list = []
        for d in range(1, degree + 1):
            temp_list.append(data_list[i][0] ** d)
        temp_data_list[i] = temp_list
    return np.array(temp_data_list)

In [7]:
def predict_zero():
    predict_zero = rdt_array.mean()

    RMSE = math.sqrt(((predict_zero - rdt_array) ** 2).sum() / len(rdt_array))
    rRMSE = RMSE / rdt_array.mean()

    return rRMSE


def predict_n(times, degree, month):
    if degree == 0:
        rRMSE_degree_month[crop + "_" + str(degree) + "_" + str(month)] = predict_zero()
        coeffs[crop + "_" + str(degree) + "_" + str(month)] = 0.0
        return

    sum_RMSE = 0
    coef = np.array([0.0 for i in range(degree)])

    for i in range(times):
        X_train, X_test, y_train, y_test = train_test_split(data_array, rdt_array, test_size=0.2)

        lr.fit(X_train, y_train)
        coef += lr.coef_
        y_predict = lr.predict(X_test)

        RMSE = math.sqrt(((y_predict - y_test) ** 2).sum() / len(y_test))
        rRMSE = RMSE / y_test.mean()

        sum_RMSE += rRMSE

    rRMSE_degree_month[crop + "_" + str(degree) + "_" + str(month)] = sum_RMSE / times
    coeffs[crop + "_" + str(degree) + "_" + str(month)] = coef / times


def find_best_degree_each_month():
    for crop in crops:
        for month in range(1, 13):
            best_degree = start
            best_RMSE = rRMSE_degree_month[crop + "_" + str(start) + "_" + str(month)]
            
            for degree in range(start + 1, end):
                if rRMSE_degree_month[crop + "_" + str(degree) + "_" + str(month)] < best_RMSE:
                    best_RMSE = rRMSE_degree_month[crop + "_" + str(degree) + "_" + str(month)]
                    best_degree = degree
            best_rRMSE_month[crop + "_" + str(best_degree) + "_" + str(month)] = best_RMSE
            best_degree_month[crop + "_" + str(month)] = best_degree

## Consider One Month

### rr24

In [23]:
start = 0
end = 10
times = 1000
total_rain = {}

coeffs = {}
rRMSE_degree_month = {}

init_x(total_rain, "rr24")

for crop in crops:
    total_rdt = {}
    init_y(total_rain)
    
    for month in range(1, 13):
        temp_data_array, rdt_array = init_list(total_rain, month)

        for d in range(start, end):
            data_array = add_degreed_data(temp_data_array, d)

            predict_n(times, d, month)

In [24]:
rRMSE_degree_month

{'OP_0_1': 0.2616310200103802,
 'OP_1_1': 0.26547931951276404,
 'OP_2_1': 0.25907479660180943,
 'OP_3_1': 0.2670972072916596,
 'OP_4_1': 0.25892090051329936,
 'OP_5_1': 0.2677465507216615,
 'OP_6_1': 0.3146549498775891,
 'OP_7_1': 0.436598726165085,
 'OP_8_1': 0.6601261769935485,
 'OP_9_1': 0.9985239152618605,
 'OP_0_2': 0.2617208144543308,
 'OP_1_2': 0.2629055108889549,
 'OP_2_2': 0.265090862164497,
 'OP_3_2': 0.2612338876962263,
 'OP_4_2': 0.2701513059162074,
 'OP_5_2': 0.34413651869862405,
 'OP_6_2': 0.388656152885267,
 'OP_7_2': 0.8054028346840729,
 'OP_8_2': 0.9362963032003571,
 'OP_9_2': 3.780362591930154,
 'OP_0_3': 0.2617208144543308,
 'OP_1_3': 0.26500830040727386,
 'OP_2_3': 0.24688670504287133,
 'OP_3_3': 0.24703701743565323,
 'OP_4_3': 0.24820136344570481,
 'OP_5_3': 0.2561808047428399,
 'OP_6_3': 0.43574695331540086,
 'OP_7_3': 0.3550448240366769,
 'OP_8_3': 0.8100859454003997,
 'OP_9_3': 2.119850852772173,
 'OP_0_4': 0.2617208144543308,
 'OP_1_4': 0.250780872192945,
 'OP_

In [25]:
best_rRMSE_month = {}
best_degree_month = {}

find_best_degree_each_month()
best_rRMSE_month

{'OP_4_1': 0.25892090051329936,
 'OP_3_2': 0.2612338876962263,
 'OP_2_3': 0.24688670504287133,
 'OP_1_4': 0.250780872192945,
 'OP_1_5': 0.2578125108355212,
 'OP_0_6': 0.2617208144543308,
 'OP_0_7': 0.2617208144543308,
 'OP_1_8': 0.2611722324334166,
 'OP_4_9': 0.2527207770542187,
 'OP_2_10': 0.25016656122473907,
 'OP_0_11': 0.2533266395078882,
 'OP_3_12': 0.2344794853366911,
 'CZH_2_1': 0.19565870709570915,
 'CZH_0_2': 0.19819454892258925,
 'CZH_4_3': 0.18391491631801243,
 'CZH_2_4': 0.1915981985218513,
 'CZH_0_5': 0.19819454892258925,
 'CZH_4_6': 0.19350541038915417,
 'CZH_2_7': 0.19523599573055245,
 'CZH_1_8': 0.19383167215022962,
 'CZH_3_9': 0.2099064440385867,
 'CZH_2_10': 0.20950004872845038,
 'CZH_0_11': 0.2119383618060273,
 'CZH_3_12': 0.2085004700677851,
 'BTH_0_1': 0.2606952441631043,
 'BTH_1_2': 0.25539537294796444,
 'BTH_4_3': 0.22379240000715478,
 'BTH_1_4': 0.23789340844720058,
 'BTH_1_5': 0.2583062174178141,
 'BTH_8_6': 0.2498918580106478,
 'BTH_6_7': 0.25679476299003895,


In [26]:
best_predict = {}
best_predict_degree = {}

for crop in crops:
    temp_list = []
    for i in range(1, 13):
        temp_list.append(best_rRMSE_month[crop + "_" + str(best_degree_month[crop + "_" + str(i)]) + "_" + str(i)])
    best_predict[crop + "_" + str(temp_list.index(min(temp_list)) + 1)] = min(temp_list)
    best_predict_degree[crop + "_" + str(temp_list.index(min(temp_list)) + 1)] = best_degree_month[crop + "_" + str(temp_list.index(min(temp_list)) + 1)]

best_predict

{'OP_12': 0.2344794853366911,
 'CZH_3': 0.18391491631801243,
 'BTH_3': 0.22379240000715478,
 'TS_6': 0.1967839701362945,
 'BTP_3': 0.2752455705091101,
 'BDP_10': 0.23107915629185072,
 'BDH_10': 0.22953832311964514,
 'OH_12': 0.21122544029714274,
 'MA_7': 0.1853970093602869}

In [27]:
best_predict_degree

{'OP_12': 3,
 'CZH_3': 4,
 'BTH_3': 4,
 'TS_6': 2,
 'BTP_3': 3,
 'BDP_10': 1,
 'BDH_10': 1,
 'OH_12': 3,
 'MA_7': 1}

In [28]:
best_coeffs = {}

for s in best_predict:
    crop, month = s.split("_")
    best_coeffs[crop + "_" + str(best_predict_degree[s]) + "_" + str(month)] = coeffs[crop + "_" + str(best_predict_degree[s]) + "_" + str(month)]
        
best_coeffs

{'OP_3_12': array([ 8.03770005e-01, -5.68744379e-03,  1.13209668e-05]),
 'CZH_4_3': array([ 6.15799413e-01, -1.14323440e-02,  8.08076278e-05, -2.04919138e-07]),
 'BTH_4_3': array([ 1.88396020e+00, -3.27269260e-02,  2.14567957e-04, -5.12348989e-07]),
 'TS_2_6': array([ 0.12397314, -0.0004549 ]),
 'BTP_3_3': array([ 1.48251403e+00, -1.94991729e-02,  6.83731424e-05]),
 'BDP_1_10': array([-0.03930852]),
 'BDH_1_10': array([-0.03944293]),
 'OH_3_12': array([ 9.00975688e-01, -6.36872761e-03,  1.24404483e-05]),
 'MA_1_7': array([0.26601264])}

## DJ_0

In [29]:
start = 0
end = 10
times = 1000
total_rad_0 = {}

coeffs = {}
rRMSE_degree_month = {}

init_x(total_rad_0, "DJ_0")

for crop in crops:
    total_rdt = {}
    init_y(total_rad_0)
    
    for month in range(1, 13):
        temp_data_array, rdt_array = init_list(total_rad_0, month)

        for d in range(start, end):
            data_array = add_degreed_data(temp_data_array, d)

            predict_n(times, d, month)

In [30]:
best_rRMSE_month = {}
best_degree_month = {}

find_best_degree_each_month()
best_rRMSE_month

{'OP_3_1': 0.25417604930541066,
 'OP_4_2': 0.25501440632576117,
 'OP_2_3': 0.25241304951091353,
 'OP_2_4': 0.24041408796543393,
 'OP_3_5': 0.23857777531451763,
 'OP_2_6': 0.2453520194519705,
 'OP_2_7': 0.24496314685012002,
 'OP_5_8': 0.2303792192730093,
 'OP_3_9': 0.22113766389052356,
 'OP_2_10': 0.23379844724577914,
 'OP_3_11': 0.24067470439445512,
 'OP_3_12': 0.24909332561040812,
 'CZH_2_1': 0.19287103278534884,
 'CZH_3_2': 0.18363724037972212,
 'CZH_2_3': 0.17989585858969262,
 'CZH_3_4': 0.18552511522908305,
 'CZH_2_5': 0.1793828961756558,
 'CZH_4_6': 0.18432250216123033,
 'CZH_2_7': 0.17646217036899828,
 'CZH_2_8': 0.16890872500066317,
 'CZH_1_9': 0.18190460523936033,
 'CZH_2_10': 0.1873348415758095,
 'CZH_4_11': 0.19496096703770316,
 'CZH_4_12': 0.2092202897072195,
 'BTH_3_1': 0.23667495077014206,
 'BTH_3_2': 0.2340578727866387,
 'BTH_2_3': 0.22026118556429386,
 'BTH_2_4': 0.2274299839300357,
 'BTH_2_5': 0.21033803189648895,
 'BTH_3_6': 0.22075342154453337,
 'BTH_2_7': 0.214320165

In [31]:
best_predict = {}
best_predict_degree = {}

for crop in crops:
    temp_list = []
    for i in range(1, 13):
        temp_list.append(best_rRMSE_month[crop + "_" + str(best_degree_month[crop + "_" + str(i)]) + "_" + str(i)])
    best_predict[crop + "_" + str(temp_list.index(min(temp_list)) + 1)] = min(temp_list)
    best_predict_degree[crop + "_" + str(temp_list.index(min(temp_list)) + 1)] = best_degree_month[crop + "_" + str(temp_list.index(min(temp_list)) + 1)]

best_predict

{'OP_9': 0.22113766389052356,
 'CZH_8': 0.16890872500066317,
 'BTH_9': 0.18304725767928406,
 'TS_3': 0.18482317411773688,
 'BTP_9': 0.25093999334063516,
 'BDP_6': 0.21725537591206287,
 'BDH_6': 0.2175221896334991,
 'OH_9': 0.16650021095660805,
 'MA_11': 0.19870155375950269}

In [32]:
best_coeffs = {}

for s in best_predict:
    crop, month = s.split("_")
    best_coeffs[crop + "_" + str(best_predict_degree[s]) + "_" + str(month)] = coeffs[crop + "_" + str(best_predict_degree[s]) + "_" + str(month)]
        
best_coeffs

{'OP_3_9': array([ 3.54646694e+00, -6.39596344e-03,  3.69048747e-06]),
 'CZH_2_8': array([ 0.12993091, -0.00013256]),
 'BTH_3_9': array([ 4.72117414e+00, -8.38414725e-03,  4.71255826e-06]),
 'TS_2_3': array([ 0.16185318, -0.00036831]),
 'BTP_3_9': array([ 5.83761254e+00, -1.03998685e-02,  5.92460963e-06]),
 'BDP_7_6': array([-1.81209431e-15, -1.03513923e-12, -3.63443926e-10, -7.05918285e-08,
         3.02340110e-10, -4.42580035e-13,  2.18812274e-16]),
 'BDH_9_6': array([-1.41444785e-26,  1.43232299e-20, -5.05001075e-21, -1.96349723e-18,
        -5.74632801e-16, -1.01422973e-13,  4.75666221e-16, -7.43288371e-19,
         3.85959194e-22]),
 'OH_4_9': array([-8.26875717e+00,  2.90588043e-02, -4.28158060e-05,  2.24485395e-08]),
 'MA_1_11': array([-0.08615323])}

## DJ_6

In [33]:
start = 0
end = 10
times = 1000
total_rad_6 = {}

coeffs = {}
rRMSE_degree_month = {}

init_x(total_rad_6, "DJ_6")

for crop in crops:
    for month in range(1, 13):
        total_rdt = {}
        init_y(total_rad_6)
        temp_data_array, rdt_array = init_list(total_rad_6, month)

        for d in range(start, end):
            data_array = add_degreed_data(temp_data_array, d)

            predict_n(times, d, month)

In [34]:
best_rRMSE_month = {}
best_degree_month = {}

find_best_degree_each_month()
best_rRMSE_month

{'OP_1_1': 0.2555472274506911,
 'OP_2_2': 0.2541728409847339,
 'OP_3_3': 0.25014729972905414,
 'OP_2_4': 0.24120999469459017,
 'OP_3_5': 0.23394061499114596,
 'OP_9_6': 0.24351208853696998,
 'OP_2_7': 0.24491708411437754,
 'OP_3_8': 0.22744224297100346,
 'OP_3_9': 0.2159460777082814,
 'OP_3_10': 0.23397701139008606,
 'OP_4_11': 0.24129827713689056,
 'OP_3_12': 0.25263627812037676,
 'CZH_2_1': 0.19196893251873973,
 'CZH_5_2': 0.1821270870524794,
 'CZH_2_3': 0.1787070829706716,
 'CZH_3_4': 0.18606680359843558,
 'CZH_4_5': 0.17326403628489245,
 'CZH_4_6': 0.1833979367471432,
 'CZH_3_7': 0.17476999526632553,
 'CZH_2_8': 0.16913705603278695,
 'CZH_1_9': 0.1814438332128339,
 'CZH_2_10': 0.1868188350174279,
 'CZH_5_11': 0.19667476857270413,
 'CZH_1_12': 0.20987918281779772,
 'BTH_1_1': 0.23744359730290385,
 'BTH_2_2': 0.2360250913008198,
 'BTH_3_3': 0.21673677603453914,
 'BTH_2_4': 0.22697429037132824,
 'BTH_3_5': 0.2055908939631606,
 'BTH_9_6': 0.2208623207446896,
 'BTH_2_7': 0.2130814819726

In [35]:
best_predict = {}
best_predict_degree = {}

for crop in crops:
    temp_list = []
    for i in range(1, 13):
        temp_list.append(best_rRMSE_month[crop + "_" + str(best_degree_month[crop + "_" + str(i)]) + "_" + str(i)])
    best_predict[crop + "_" + str(temp_list.index(min(temp_list)) + 1)] = min(temp_list)
    best_predict_degree[crop + "_" + str(temp_list.index(min(temp_list)) + 1)] = best_degree_month[crop + "_" + str(temp_list.index(min(temp_list)) + 1)]

best_predict

{'OP_9': 0.2159460777082814,
 'CZH_8': 0.16913705603278695,
 'BTH_9': 0.17852064447385022,
 'TS_3': 0.18461378955068078,
 'BTP_9': 0.24710222716544128,
 'BDP_9': 0.2187693938868676,
 'BDH_9': 0.21923613645142154,
 'OH_9': 0.1646096279810979,
 'MA_8': 0.19817057735579222}

In [36]:
best_coeffs = {}

for s in best_predict:
    crop, month = s.split("_")
    best_coeffs[crop + "_" + str(best_predict_degree[s]) + "_" + str(month)] = coeffs[crop + "_" + str(best_predict_degree[s]) + "_" + str(month)]
        
best_coeffs

{'OP_3_9': array([ 2.20620868e+00, -6.04853172e-03,  5.15118102e-06]),
 'CZH_2_8': array([ 0.07203989, -0.00012345]),
 'BTH_3_9': array([ 2.89505290e+00, -7.81887492e-03,  6.45110933e-06]),
 'TS_2_3': array([ 0.05086313, -0.00048099]),
 'BTP_3_9': array([ 3.54497059e+00, -9.57044601e-03,  8.01807008e-06]),
 'BDP_2_9': array([ 0.29463332, -0.00048415]),
 'BDH_2_9': array([ 0.29574047, -0.00048591]),
 'OH_3_9': array([ 2.59865989e+00, -7.10341846e-03,  5.95432094e-06]),
 'MA_1_8': array([-0.09332734])}